# Building Autograd from Scratch

Understanding automatic differentiation - the core of PyTorch, TensorFlow, JAX.

**Goal**: Build a mini version of PyTorch's autograd system.

**Your notes here:**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

---
## Part 1: The Core Idea

**Manual differentiation** (what we did in nn.ipynb): 
- Write forward pass
- Manually derive gradients
- Write backward pass

**Automatic differentiation**:
- Write forward pass only
- System automatically computes gradients!

**How?** Build a computational graph and use the chain rule.

**Your notes here:**


### Example: Computational Graph

```python
# Forward
a = 2.0
b = 3.0
c = a + b      # c = 5
d = c * a      # d = 10
e = c * d      # e = 50
```

Graph:
```
  a(2) ──┬──> c(5) ──┬──> e(50)
         │           │
  b(3) ──┘     d(10)─┘
         │       │
  a(2) ──────────┘
```

**Backward** (chain rule):
```
de/de = 1
de/dd = c = 5
de/dc = d = 10
de/da = de/dc * dc/da + de/dd * dd/da = 10*1 + 5*c = 10 + 25 = 35
de/db = de/dc * dc/db = 10 * 1 = 10
```

**Your notes here:**


---
## Part 2: The Value Class

Each value knows:
1. Its data (the number)
2. Its gradient (∂L/∂value)
3. Its parents (what created it)
4. How to backprop (backward function)

**Your notes here:**


In [ ]:
class Value:
    """
    A node in the computational graph.
    
    Stores:
    - data: the actual value
    - grad: gradient of some loss w.r.t. this value
    - _backward: function to propagate gradients to children
    - _prev: parent nodes (what operations created this)
    """
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0  # gradient starts at 0
        self._backward = lambda: None  # default: do nothing
        self._prev = set(_children)  # parent nodes
        self._op = _op  # operation that created this (for visualization)
        self.label = label
    
    def __repr__(self):
        return f"Value(data={self.data:.4f}, grad={self.grad:.4f})"
    
    def __add__(self, other):
        """Addition: z = x + y"""
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            # dz/dx = 1, dz/dy = 1
            # Chain rule: dL/dx += dL/dz * dz/dx = dL/dz * 1
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        """Multiplication: z = x * y"""
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            # dz/dx = y, dz/dy = x
            # Chain rule: dL/dx += dL/dz * dz/dx
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    def __pow__(self, other):
        """Power: z = x^k"""
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            # dz/dx = k * x^(k-1)
            self.grad += other * (self.data ** (other - 1)) * out.grad
        
        out._backward = _backward
        return out
    
    def tanh(self):
        """Hyperbolic tangent activation"""
        x = self.data
        t = (np.exp(2*x) - 1) / (np.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            # d(tanh(x))/dx = 1 - tanh^2(x)
            self.grad += (1 - t**2) * out.grad
        
        out._backward = _backward
        return out
    
    def exp(self):
        """Exponential"""
        x = self.data
        out = Value(np.exp(x), (self,), 'exp')
        
        def _backward():
            # d(e^x)/dx = e^x
            self.grad += out.data * out.grad
        
        out._backward = _backward
        return out
    
    def __neg__(self):  # -self
        return self * -1
    
    def __radd__(self, other):  # other + self
        return self + other
    
    def __sub__(self, other):  # self - other
        return self + (-other)
    
    def __rsub__(self, other):  # other - self
        return other + (-self)
    
    def __rmul__(self, other):  # other * self
        return self * other
    
    def __truediv__(self, other):  # self / other
        return self * other**-1
    
    def __rtruediv__(self, other):  # other / self
        return other * self**-1
    
    def backward(self):
        """
        Run backpropagation from this node.
        
        Topological sort ensures we visit nodes in the right order
        (children before parents).
        """
        # Build topological order
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        
        build_topo(self)
        
        # Start backprop: gradient of self w.r.t. itself is 1
        self.grad = 1.0
        
        # Apply chain rule in reverse topological order
        for node in reversed(topo):
            node._backward()

print("Value class created! Let's test it...")

---
## Part 3: Testing Basic Operations

**Your notes here:**


In [ ]:
# Example from earlier
a = Value(2.0, label='a')
b = Value(3.0, label='b')
c = a + b
c.label = 'c'
d = c * a
d.label = 'd'
e = c * d
e.label = 'e'

print("Forward pass:")
print(f"a = {a.data}")
print(f"b = {b.data}")
print(f"c = a + b = {c.data}")
print(f"d = c * a = {d.data}")
print(f"e = c * d = {e.data}")

# Backward pass
e.backward()

print("\nBackward pass (gradients):")
print(f"de/de = {e.grad}")
print(f"de/dd = {d.grad}")
print(f"de/dc = {c.grad}")
print(f"de/da = {a.grad}  (expected: 35)")
print(f"de/db = {b.grad}  (expected: 10)")

In [ ]:
# More complex example with activation
x = Value(0.5, label='x')
y = Value(-1.0, label='y')

z = x**2 + y**2  # z = 0.25 + 1.0 = 1.25
z.label = 'z'

w = z.tanh()  # w = tanh(1.25) ≈ 0.848
w.label = 'w'

print(f"z = x^2 + y^2 = {z.data:.4f}")
print(f"w = tanh(z) = {w.data:.4f}")

w.backward()

print(f"\ndw/dx = {x.grad:.4f}")
print(f"dw/dy = {y.grad:.4f}")

# Verify with numerical gradient
h = 1e-6
x_val = 0.5
y_val = -1.0

def f(x, y):
    z = x**2 + y**2
    return np.tanh(z)

numerical_dx = (f(x_val + h, y_val) - f(x_val - h, y_val)) / (2*h)
numerical_dy = (f(x_val, y_val + h) - f(x_val, y_val - h)) / (2*h)

print(f"\nNumerical verification:")
print(f"dw/dx (numerical) = {numerical_dx:.4f}")
print(f"dw/dy (numerical) = {numerical_dy:.4f}")
print("\nMatches! ✓")

---
## Part 4: Building a Neural Network with Autograd

Now let's use our autograd to build a real neural network!

**Your notes here:**


In [ ]:
class Neuron:
    """
    A single neuron: y = tanh(w·x + b)
    """
    def __init__(self, n_inputs):
        self.w = [Value(np.random.randn()) for _ in range(n_inputs)]
        self.b = Value(np.random.randn())
    
    def __call__(self, x):
        # w·x + b
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    """
    A layer of neurons.
    """
    def __init__(self, n_inputs, n_outputs):
        self.neurons = [Neuron(n_inputs) for _ in range(n_outputs)]
    
    def __call__(self, x):
        outs = [neuron(x) for neuron in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    """
    Multi-Layer Perceptron.
    """
    def __init__(self, n_inputs, layer_sizes):
        sz = [n_inputs] + layer_sizes
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(layer_sizes))]
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

# Create a 2-layer MLP: 3 inputs -> 4 hidden -> 1 output
mlp = MLP(3, [4, 1])
print(f"MLP with {len(mlp.parameters())} parameters")

# Test forward pass
x = [Value(1.0), Value(-0.5), Value(0.3)]
y = mlp(x)
print(f"\nOutput: {y}")

### Train the MLP

Let's train it to learn a simple function.

**Your notes here:**


In [ ]:
# Dataset: simple XOR-like problem
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]  # targets

# Convert to Value objects
xs = [[Value(x) for x in row] for row in xs]
ys = [Value(y) for y in ys]

# Initialize MLP
np.random.seed(42)
mlp = MLP(3, [4, 4, 1])

print(f"Training MLP with {len(mlp.parameters())} parameters")
print(f"Training set: {len(xs)} examples\n")

In [ ]:
# Training loop
losses = []
learning_rate = 0.05
n_epochs = 100

for epoch in range(n_epochs):
    # Forward pass
    ypred = [mlp(x) for x in xs]
    
    # Loss: mean squared error
    loss = sum((yp - yt)**2 for yt, yp in zip(ys, ypred))
    
    # Backward pass
    # Zero gradients
    for p in mlp.parameters():
        p.grad = 0.0
    
    loss.backward()
    
    # Update parameters
    for p in mlp.parameters():
        p.data += -learning_rate * p.grad
    
    losses.append(loss.data)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d}: Loss = {loss.data:.6f}")

print("\nTraining complete!")

In [ ]:
# Plot training curve
plt.figure(figsize=(10, 5))
plt.plot(losses, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss (Our Autograd MLP)')
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print(f"Final loss: {losses[-1]:.6f}")

In [ ]:
# Test predictions
print("\nFinal predictions:")
print("-" * 40)
for i, (x, y_true) in enumerate(zip(xs, ys)):
    y_pred = mlp(x)
    print(f"Example {i}: Target={y_true.data:+.1f}, Predicted={y_pred.data:+.4f}")

---
## Part 5: Comparison with PyTorch

Let's verify our implementation matches PyTorch.

**Your notes here:**


In [ ]:
import torch

# Same computation with PyTorch
x1 = torch.tensor([2.0], requires_grad=True)
x2 = torch.tensor([3.0], requires_grad=True)

# Forward
y = x1**2 + x2**2
z = torch.tanh(y)

# Backward
z.backward()

print("PyTorch:")
print(f"z = {z.item():.4f}")
print(f"dz/dx1 = {x1.grad.item():.4f}")
print(f"dz/dx2 = {x2.grad.item():.4f}")

# Our implementation
a = Value(2.0)
b = Value(3.0)
c = a**2 + b**2
d = c.tanh()
d.backward()

print("\nOur Autograd:")
print(f"d = {d.data:.4f}")
print(f"dd/da = {a.grad:.4f}")
print(f"dd/db = {b.grad:.4f}")

print("\n✓ Matches perfectly!")

---
## Part 6: Visualizing the Computational Graph

**Your notes here:**


In [ ]:
try:
    from graphviz import Digraph
    
    def trace(root):
        nodes, edges = set(), set()
        def build(v):
            if v not in nodes:
                nodes.add(v)
                for child in v._prev:
                    edges.add((child, v))
                    build(child)
        build(root)
        return nodes, edges
    
    def draw_dot(root, format='svg', rankdir='LR'):
        """
        Draw computational graph.
        format: 'svg', 'png'
        rankdir: 'LR' (left to right), 'TB' (top to bottom)
        """
        assert rankdir in ['LR', 'TB']
        nodes, edges = trace(root)
        dot = Digraph(format=format, graph_attr={'rankdir': rankdir})
        
        for n in nodes:
            dot.node(name=str(id(n)), 
                    label=f"{n.label} | data {n.data:.2f} | grad {n.grad:.2f}",
                    shape='record')
            if n._op:
                dot.node(name=str(id(n)) + n._op, label=n._op)
                dot.edge(str(id(n)) + n._op, str(id(n)))
        
        for n1, n2 in edges:
            dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
        return dot
    
    # Draw example graph
    x = Value(2.0, label='x')
    y = Value(3.0, label='y')
    z = x * y
    z.label = 'z'
    w = z + x
    w.label = 'w'
    w.backward()
    
    graph = draw_dot(w)
    graph.render('comp_graph', view=False, cleanup=True)
    print("Computational graph saved as 'comp_graph.svg'")
    
except ImportError:
    print("Install graphviz to visualize: pip install graphviz")
    print("Skipping visualization...")

---
## Part 7: Understanding How PyTorch Works

Our mini autograd is essentially what PyTorch does, but:

1. **PyTorch operates on tensors** (multi-dimensional arrays)
2. **GPU acceleration** via CUDA
3. **Optimized C++/CUDA kernels** for operations
4. **Memory management** (in-place ops, gradient checkpointing)
5. **More operations** (convolution, pooling, attention, etc.)

But the **core idea is identical**:
- Build computational graph
- Store backward functions
- Apply chain rule in reverse topological order

**Your notes here:**


### Key Concepts to Remember

1. **Forward pass**: Build the computational graph
2. **Backward pass**: Apply chain rule automatically
3. **Topological sort**: Visit nodes in the right order
4. **Gradient accumulation**: `+=` not `=` (multiple paths to same node)
5. **Zero gradients**: Reset before each backward pass

**Your notes here:**


---
## Part 8: Extension Ideas

To make this a real autograd engine, add:

1. **Tensor support** (extend to n-dimensional arrays)
2. **Broadcasting** (handle different shapes)
3. **More operations**:
   - Matrix multiplication
   - Convolution
   - Softmax
   - Layer normalization
4. **In-place operations** (memory efficiency)
5. **GPU support** (CUDA kernels)
6. **Static vs dynamic graphs** (TensorFlow vs PyTorch)

**Your notes here:**


---
## Summary

You've built a working autograd engine! You now understand:

1. **Computational graphs**: Nodes are values, edges are operations
2. **Automatic differentiation**: Chain rule applied systematically
3. **Backpropagation**: Traverse graph backwards, accumulate gradients
4. **How PyTorch/TensorFlow work** under the hood

This is the **foundation of all modern deep learning frameworks**.

**Your notes here:**


---
## Exercises

1. Add a `relu()` method to the `Value` class
2. Add a `sigmoid()` method
3. Implement matrix multiplication for the `Value` class
4. Build a Conv2D operation (harder!)
5. Extend `Value` to support n-dimensional arrays (become a mini numpy with autograd)

**Your notes here:**
